In [ ]:
!wget -O /content/drive/MyDrive/VDSet_and_VCSet.rar "https://zenodo.org/records/14792742/files/VDSet_and_VCSet.rar?download=1"

--2025-03-30 10:54:43--  https://zenodo.org/records/14792742/files/VDSet_and_VCSet.rar?download=1
Resolving zenodo.org (zenodo.org)... 188.185.48.194, 188.185.45.92, 188.185.43.25, ...
Connecting to zenodo.org (zenodo.org)|188.185.48.194|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4092592108 (3.8G) [application/octet-stream]
Saving to: ‘/content/drive/MyDrive/VDSet_and_VCSet.rar’

/content/drive/MyDr 100%[===================>]   3.81G  13.7MB/s    in 4m 55s  

2025-03-30 10:59:39 (13.2 MB/s) - ‘/content/drive/MyDrive/VDSet_and_VCSet.rar’ saved [4092592108/4092592108]



In [ ]:
!apt-get install unrar -y


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newest version (1:6.1.5-1ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [ ]:
!mkdir /content/drive/MyDrive/extracted_files

In [ ]:
!unrar x /content/drive/MyDrive/VDSet_and_VCSet.rar /content/drive/MyDrive/extracted_files/

Streaming output truncated to the last 5000 lines.
Extracting  /content/drive/MyDrive/extracted_files/2_Datasets/Vehicle Detection Dataset (VDset)/images/Frame367.png      48%  OK 
Extracting  /content/drive/MyDrive/extracted_files/2_Datasets/Vehicle Detection Dataset (VDset)/images/Frame367.xml      48%  OK 
Extracting  /content/drive/MyDrive/extracted_files/2_Datasets/Vehicle Detection Dataset (VDset)/images/Frame368.png      48%  OK 
Extracting  /content/drive/MyDrive/extracted_files/2_Datasets/Vehicle Detection Dataset (VDset)/images/Frame368.xml      48%  OK 
Extracting  /content/drive/MyDrive/extracted_files/2_Datasets/Vehicle Detection Dataset (VDset)/images/Frame369.png      48%  OK 
Extracting  /content/drive/MyDrive/extracted_files/2_Datasets/Vehicle Detection Dataset (VDset)/images/Frame369.xml      48%  OK 
Extracting  /content/drive/MyDrive/extracted_files/2_Datasets/Vehicle Detection Dataset (VDset)/images/Frame37.png 

In [ ]:
!ls /content/drive/MyDrive/extracted_files/2_Datasets/'Vehicle Detection Dataset (VDset)'/images

 dusukFrame100.png   Frame87.xml
 dusukFrame100.xml   Frame88.png
 dusukFrame101.png   Frame88.xml
 dusukFrame101.xml   Frame89.png
 dusukFrame102.png   Frame89.xml
 dusukFrame102.xml   Frame8.png
 dusukFrame103.png   Frame8.xml
 dusukFrame103.xml   Frame90.png
 dusukFrame104.png   Frame90.xml
 dusukFrame104.xml   Frame91.png
 dusukFrame105.png   Frame91.xml
 dusukFrame105.xml   Frame92.png
 dusukFrame106.png   Frame92.xml
 dusukFrame106.xml   Frame93.png
 dusukFrame107.png   Frame93.xml
 dusukFrame107.xml   Frame94.png
 dusukFrame108.png   Frame94.xml
 dusukFrame108.xml   Frame95.png
 dusukFrame109.png   Frame95.xml
 dusukFrame109.xml   Frame96.png
 dusukFrame10.png    Frame96.xml
 dusukFrame10.xml    Frame97.png
 dusukFrame110.png   Frame97.xml
 dusukFrame110.xml   Frame98.png
 dusukFrame111.png   Frame98.xml
 dusukFrame111.xml   Frame99.png
 dusukFrame112.png   Frame99.xml
 dusukFrame112.xml   Frame9.png
 dusukFrame113.png   Frame9.xml
 dusukFrame113.xml   Qualityframe143099.png
 du

In [ ]:
!rm -r /content/drive/MyDrive/extracted_files/2_Datasets/'Vehicle Classification Dataset (VCset)'

In [ ]:
from google.colab import files


In [ ]:


# Correctly formatted file path
file_path = "/content/drive/MyDrive/extracted_files/2_Datasets/Vehicle Detection Dataset (VDset)/images/dusukFrame100.xml"

# Download the file
files.download(file_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import os
import xml.etree.ElementTree as ET
import cv2
import multiprocessing
import time

def convert_xml_to_yolo(xml_path, class_names, output_path, image_width, image_height):
    """Converts a single Pascal VOC XML annotation to YOLO format."""
    try:
        tree = ET.parse(xml_path)
        root = tree.getroot()

        image_name = root.find('filename').text
        txt_file_name = os.path.splitext(image_name)[0] + '.txt'
        txt_file_path = os.path.join(output_path, txt_file_name)

        with open(txt_file_path, 'w') as txt_file:
            for obj in root.findall('object'):
                class_name = obj.find('name').text
                if class_name not in class_names:
                    continue

                class_id = class_names.index(class_name)

                bndbox = obj.find('bndbox')
                xmin = int(bndbox.find('xmin').text)
                ymin = int(bndbox.find('ymin').text)
                xmax = int(bndbox.find('xmax').text)
                ymax = int(bndbox.find('ymax').text)

                x_center = (xmin + xmax) / (2 * image_width)
                y_center = (ymin + ymax) / (2 * image_height)
                width = (xmax - xmin) / image_width
                height = (ymax - ymin) / image_height

                txt_file.write(f"{class_id} {x_center} {y_center} {width} {height}\n")
        return True

    except FileNotFoundError:
        print(f"Error: XML file not found: {xml_path}")
        return False
    except Exception as e:
        print(f"Error processing XML file {xml_path}: {e}")
        return False

def process_image(image_file, image_folder, annotation_folder, output_folder, class_names):
    """Processes a single image and its corresponding XML annotation."""
    image_path = os.path.join(image_folder, image_file)
    xml_file = os.path.splitext(image_file)[0] + '.xml'
    xml_path = os.path.join(annotation_folder, xml_file)

    try:
        img = cv2.imread(image_path)
        image_height, image_width = img.shape[:2]
    except Exception as e:
        print(f"Error reading image {image_file}: {e}")
        return False

    return convert_xml_to_yolo(xml_path, class_names, output_folder, image_width, image_height)

if __name__ == "__main__":
    # --- Configuration ---
    image_folder = "/content/drive/MyDrive/extracted_files/2_Datasets/Vehicle Detection Dataset (VDset)/images/"  # Path to your image folder
    annotation_folder = image_folder  # Assuming XML files are in the same folder
    output_folder = "/content/drive/MyDrive/yolo_labels/"  # Where to save the YOLO txt files
    class_names = ['bus', 'car', 'motorcycle', 'truck', 'van']  # Your vehicle class names

    # --- Setup ---
    os.makedirs(output_folder, exist_ok=True)
    image_files = [f for f in os.listdir(image_folder) if f.endswith('.png')]

    # --- Parallel Processing ---
    start_time = time.time()
    num_processes = multiprocessing.cpu_count()  # Use all available CPU cores
    pool = multiprocessing.Pool(processes=num_processes)

    results = [pool.apply_async(process_image, args=(image_file, image_folder, annotation_folder, output_folder, class_names))
               for image_file in image_files]

    pool.close()
    pool.join()  # Wait for all processes to complete

    success_count = sum(res.get() for res in results if res.get())
    failure_count = len(image_files) - success_count

    end_time = time.time()
    elapsed_time = end_time - start_time

    print(f"Successfully converted {success_count} XML files.")
    print(f"Failed to convert {failure_count} XML files.")
    print(f"Total processing time: {elapsed_time:.2f} seconds")


Successfully converted 4291 XML files.
Failed to convert 0 XML files.
Total processing time: 509.92 seconds


In [ ]:
!ls /content/drive/MyDrive/yolo_labels

 dusukFrame100.txt   Frame88.txt
 dusukFrame101.txt   Frame89.txt
 dusukFrame102.txt   Frame8.txt
 dusukFrame103.txt   Frame90.txt
 dusukFrame104.txt   Frame91.txt
 dusukFrame105.txt   Frame92.txt
 dusukFrame106.txt   Frame93.txt
 dusukFrame107.txt   Frame94.txt
 dusukFrame108.txt   Frame95.txt
 dusukFrame109.txt   Frame96.txt
 dusukFrame10.txt    Frame97.txt
 dusukFrame110.txt   Frame98.txt
 dusukFrame111.txt   Frame99.txt
 dusukFrame112.txt   Frame9.txt
 dusukFrame113.txt   Qualityframe143099.txt
 dusukFrame114.txt   Qualityframe143174.txt
 dusukFrame115.txt   Qualityframe143249.txt
 dusukFrame116.txt   Qualityframe143324.txt
 dusukFrame117.txt   Qualityframe143399.txt
 dusukFrame118.txt   Qualityframe143474.txt
 dusukFrame119.txt   Qualityframe143549.txt
 dusukFrame11.txt    Qualityframe143624.txt
 dusukFrame120.txt   Qualityframe143699.txt
 dusukFrame121.txt   Qualityframe143774.txt
 dusukFrame122.txt   Qualityframe144224.txt
 dusukFrame123.txt   Qualityframe144299.txt
 dusukFrame1

In [ ]:
import os
import random
import shutil  # Import the shutil module

# Define paths
images_dir = "/content/drive/MyDrive/extracted_files/2_Datasets/Vehicle Detection Dataset (VDset)/images"
labels_dir = "/content/drive/MyDrive/yolo_labels"
output_dir = "/content/yolov8_data"

# Create the output directory structure
os.makedirs(os.path.join(output_dir, "images", "train"), exist_ok=True)
os.makedirs(os.path.join(output_dir, "images", "val"), exist_ok=True)
os.makedirs(os.path.join(output_dir, "labels", "train"), exist_ok=True)
os.makedirs(os.path.join(output_dir, "labels", "val"), exist_ok=True)

# Get all image and label files
image_files = [f for f in os.listdir(images_dir) if f.endswith(".png")]
label_files = [f for f in os.listdir(labels_dir) if f.endswith(".txt")]

# Ensure that each image has a corresponding label file
image_names = [os.path.splitext(f)[0] for f in image_files]
label_names = [os.path.splitext(f)[0] for f in label_files]

if set(image_names) != set(label_names):
    print("Error: Mismatch between image and label files.")
    exit()

# Split the files into training and validation sets (80/20 split)
random.shuffle(image_files)
train_size = int(0.8 * len(image_files))
train_images = image_files[:train_size]
val_images = image_files[train_size:]

# Move the files to the appropriate directories
def move_files(file_list, source_dir, dest_dir, subfolder):
    for file in file_list:
        source_path = os.path.join(source_dir, file)
        dest_path = os.path.join(dest_dir, subfolder, file)
        shutil.move(source_path, dest_path)  # Use shutil.move

move_files(train_images, images_dir, output_dir, "images/train")
move_files(val_images, images_dir, output_dir, "images/val")

# Move the corresponding label files
train_labels = [f"{os.path.splitext(f)[0]}.txt" for f in train_images]
val_labels = [f"{os.path.splitext(f)[0]}.txt" for f in val_images]

move_files(train_labels, labels_dir, output_dir, "labels/train")
move_files(val_labels, labels_dir, output_dir, "labels/val")

print("Data preparation complete.")


Data preparation complete.


In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.0/950.0 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 39.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [ ]:
!cp -r /content/yolov8_data /content/drive/MyDrive/yolov8_data/

In [ ]:
import os

data_yaml_path = "/content/yolov8_data/data.yaml"
data_yaml_content = """
train: /content/yolov8_data/images/train
val: /content/yolov8_data/images/val

nc: 5  # Number of classes
names: ['bus', 'car', 'motorcycle', 'truck', 'van']  # Class names
"""

with open(data_yaml_path, "w") as f:
    f.write(data_yaml_content)

print(f"Created data.yaml at {data_yaml_path}")


Created data.yaml at /content/yolov8_data/data.yaml


In [ ]:
!rm -r /content/drive/MyDrive/yolov8_data

In [ ]:
!cp -r /content/yolov8_data /content/drive/MyDrive/

In [ ]:
files.download('/content/drive/MyDrive/yolov8_data')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>